In [ ]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


class ArithmeticDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = f"Translate: {row['Description']} {row['Question']}"
        target_text = row["Equation"]
        
        inputs = self.tokenizer(input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": targets["input_ids"].squeeze(0),
        }

df = pd.read_csv("ArithOpsTrain.csv")
df_train, df_val = train_test_split(df, test_size=0.1, random_state=42)

tokenizer = T5Tokenizer.from_pretrained("t5-small")

train_dataset = ArithmeticDataset(df_train, tokenizer)
val_dataset = ArithmeticDataset(df_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)



device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

def train_model(model, train_loader, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} Loss: {avg_loss:.4f}")

train_model(model, train_loader)

# Save model
model.save_pretrained("t5_arithmetic_model")
tokenizer.save_pretrained("t5_arithmetic_model")


def predict_equation(description, question):
    model.eval()
    input_text = f"Translate: {description} {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)
    
    with torch.no_grad():
        output = model.generate(**inputs, max_length=50)
    
    equation = tokenizer.decode(output[0], skip_special_tokens=True)
    return equation


test_df = pd.read_csv("ArithOpsTest.csv") 
test_df["Predicted Equation"] = test_df.apply(lambda row: predict_equation(row["Description"], row["Question"]), axis=1)


def evaluate_expression(expression, input_numbers):
    try:
        formatted_expr = expression
        for i, num in enumerate(eval(input_numbers)):  
            formatted_expr = formatted_expr.replace(f"num{i+1}", str(num))
        return eval(formatted_expr)  
    except Exception as e:
        print(f"Error in evaluating: {expression} -> {e}")
        return None

test_df["Final Output"] = test_df.apply(lambda row: evaluate_expression(row["Predicted Equation"], row["Input Numbers"]), axis=1)



output_df = test_df[["Description", "Question", "Final Output"]]
output_df.to_csv("ArithmeticTestResults.csv", index=False)
